In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

In [ ]:
class Strategy():
    def __init__(self, client, symbols, timeframe, startdate = "2 year ago UTC", service_charge = 0.00075, money = 1000, storage = 0):
        self.client = client
        self.symbols = symbols
        self.timeframe = timeframe
        self.startdate = startdate
        self.service_charge = service_charge
        self.start_money = [money] * len(symbols)
        self.start_storage = [storage] * len(symbols)

        self.all_data = []
        for coin in symbols:
            bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeframe,start_str=self.startdate)
            test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
            test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
            test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
            test_df["open"] = pd.to_numeric(test_df["open"])
            test_df["high"] = pd.to_numeric(test_df["high"])
            test_df["low"] = pd.to_numeric(test_df["low"])
            test_df["close"] = pd.to_numeric(test_df["close"])
            test_df["volume"] = pd.to_numeric(test_df["volume"])
            test_df["signal"] = ["None"] * len(test_df)
            self.all_data.append(test_df)

        self.profit = [0] * len(symbols)
    def print(self):
        print("Time frame", self.timeframe)
        print("Start from", self.startdate)

        for i in range(len(self.symbols)):
            print("=============")
            print(self.symbols[i], self.profit[i])

        print("=============")
        print("average", sum(self.profit) / len(self.profit))

    def decide_action(self):
        pass

    def back_test(self, type = "all in", fixed_money = None):
        for i in range(len(self.symbols)):
            money = self.start_money[i]
            storage = self.start_storage[i]
            for j in range(len(self.all_data[i])):
                if (self.all_data[i]["signal"][j] == "buy"):
                    if (type == "all in"):
                        if (storage < 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += money / self.all_data[i]["close"][j] 
                        money = 0
                    else:
                        if (storage > 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += fixed_money[i] / self.all_data[i]["close"][j] 
                        money -= fixed_money[i]
                elif (self.all_data[i]["signal"][j] == "sell"):
                    if (type == "all in"):
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= money / self.all_data[i]["close"][j] 
                        money += money
                    else:
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= fixed_money[i] / self.all_data[i]["close"][j] 
                        money += fixed_money[i]
            self.profit[i] = (money + storage * self.all_data[i]["close"][len(self.all_data[i]) - 1]) / self.start_money[i]

In [ ]:
class EMA(Strategy):
    def decide_action(self, smooth = 2, short_period = 10, long_period = 100, stable_offset = 150):
        for i in range(len(self.all_data)):
            self.all_data[i]["short EMA"] = self.all_data[i]["close"].copy()
            self.all_data[i]["long EMA"] = self.all_data[i]["close"].copy()
            self.all_data[i]["signal"] = ["None"] * len(self.all_data[i])

            for j in range(1, len(self.all_data[i])):
                self.all_data[i]["short EMA"][j] = self.all_data[i]["short EMA"][j] * smooth / (short_period + 1) + self.all_data[i]["short EMA"][j - 1] * (1 - smooth / (short_period + 1))
                self.all_data[i]["long EMA"][j] = self.all_data[i]["long EMA"][j] * smooth / (long_period + 1) + self.all_data[i]["long EMA"][j - 1] * (1 - smooth / (long_period + 1))
            for j in range(stable_offset, len(self.all_data[i])):
                if (self.all_data[i]["short EMA"][j - 1] > self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] < self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "buy"
                elif (self.all_data[i]["short EMA"][j - 1] < self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] > self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "sell"


In [ ]:
profits = []
count = 0
target_symbol = ["BTC", "ETH", "ADA", "BNB"]
time_frames = ["1d", "12h", "6h", "4h", "2h", "1h"]
time_frames = ["12h"]
start_str = "1 year ago UTC"
client = Client("YjY0w2BJZZ35o2Q9So1OvRP1gm3exc01LKp2yyLpMth5xmqVn0tQosRYOPrnb7la", "5rRXJ2rOa96hVPZTTOfVP9H4blP6AMt2QN2M0RWX2cMq9PngbJ5MEK88eGB9I4YD")
res = []
for timeframe in time_frames:
    a = EMA(client, target_symbol, timeframe, start_str)
    b = EMA(client, target_symbol, timeframe, start_str)
    for short_period in range(5, 30):
        for long_period in range(50,150):
            a.decide_action(short_period=short_period, long_period=long_period, stable_offset=long_period)
            b.decide_action(short_period=short_period, long_period=long_period, stable_offset=long_period)
            a.back_test(type="fixed_money", fixed_money=[1000] * len(target_symbol))
            b.back_test(type="all in", fixed_money=[1000] * len(target_symbol))
            res.append([short_period, long_period, timeframe, sum(a.profit) / len(a.profit)])
            print("short", short_period, "long", long_period, "timeframe", timeframe, "average","fixed money", sum(a.profit) / len(a.profit), "all in", sum(b.profit) / len(b.profit))

In [ ]:
a.print()